<a href="https://colab.research.google.com/github/jfogarty/machine-learning-intro-workshop/blob/master/misc/pyspark_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running Pyspark in Colab

To run spark in Colab, first we need to install all the dependencies in Colab environment such as Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark in order to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

**Note!** This was out of date and had to be updated from [apache spark](https://www-us.apache.org/dist/spark) to 2.4.3 form 2.4.1 before it would install.

Now that we have installed Spark and Java in Colab, it is time to set the environment path that enables us to run PySpark in our Colab environment. Set the location of Java and Spark by running the following code:


In [0]:
import os
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SPARK_HOME = "/content/spark-2.4.3-bin-hadoop2.7"

def set_os_environ_path(var, val):
    os.environ[var] = val
    if not os.path.exists(JAVA_HOME):
        print(f"** Yikes! the {var} path {val} does not exist!  Your environment is not valid.")

set_os_environ_path("JAVA_HOME",  JAVA_HOME)
set_os_environ_path("SPARK_HOME", SPARK_HOME)


**Note!** You **must** check these paths in the **Files** tab on the left side of your notebook page.  

We can run a local spark session to test our installation:

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Linear Regression Model

Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. For example, a modeler might want to predict the forecast of the rain based on the humidity ratio. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line. Detailed about linear regression can be found here.

For our purpose of starting with Pyspark in Colab and to keep things simple, we will use the famous Boston Housing dataset. A full description of this dataset can be found in this [link](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html).

### The Boston Housing Dataset

A Dataset derived from information collected by the U.S. Census Service concerning housing in the area of Boston Mass.
BackUpDelve

This dataset contains information collected by the U.S Census Service concerning housing in the area of Boston Mass. It was obtained from the [StatLib archive](http://lib.stat.cmu.edu/datasets/boston), and has been used extensively throughout the literature to benchmark algorithms. However, these comparisons were primarily done outside of Delve and are thus somewhat suspect. The dataset is small in size with only 506 cases.

The data was originally published by Harrison, D. and Rubinfeld, D.L. `Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.

#### Dataset Naming
The name for this dataset is simply boston. It has two prototasks: nox, in which the nitrous oxide level is to be predicted; and price, in which the median value of a home is to be predicted

#### Miscellaneous Details

- Origin : The origin of the boston housing data is Natural.
- Usage : This dataset may be used for Assessment.
- Number of Cases : The dataset contains a total of 506 cases.
- Order : The order of the cases is mysterious.
- Variables : There are 14 attributes in each case of the dataset. They are:
  - CRIM - per capita crime rate by town
  - ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
  - INDUS - proportion of non-retail business acres per town.
  - CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
  - NOX - nitric oxides concentration (parts per 10 million)
  - RM - average number of rooms per dwelling
  - AGE - proportion of owner-occupied units built prior to 1940
  - DIS - weighted distances to five Boston employment centres
  - RAD - index of accessibility to radial highways
  - TAX - full-value property-tax rate per 10,000 dollars
  - PTRATIO - pupil-teacher ratio by town
  - B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
  - LSTAT - % lower status of the population
  - MEDV - Median value of owner-occupied homes in $1000's

- Note : Variable #14 seems to be censored at 50.00 \(corresponding to a median price of 50,000 dollars);

 Censoring is suggested by the fact that the highest median price of exactly 50,000 dollars is reported in 16 cases, while 15 cases have prices between 40,000 dollars  and 50,000 dollars , with prices rounded to the nearest hundred. Harrison and Rubinfeld do not mention any censoring.

### End of notebook.